# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe1d44e0b50>
├── 'a' --> tensor([[ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890]])
└── 'x' --> <FastTreeValue 0x7fe1d44d58b0>
    └── 'c' --> tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                        [ 0.7860,  0.4375,  0.5272, -1.3228],
                        [ 1.3343, -0.8959,  1.2044,  2.3078]])

In [4]:
t.a

tensor([[ 1.1956, -0.6899, -0.1991],
        [-0.3943, -0.8755,  0.3890]])

In [5]:
%timeit t.a

73.3 ns ± 0.0754 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe1d44e0b50>
├── 'a' --> tensor([[ 0.7978,  0.1948, -1.2163],
│                   [ 1.9775,  0.0574, -1.1254]])
└── 'x' --> <FastTreeValue 0x7fe1d44d58b0>
    └── 'c' --> tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                        [ 0.7860,  0.4375,  0.5272, -1.3228],
                        [ 1.3343, -0.8959,  1.2044,  2.3078]])

In [7]:
%timeit t.a = new_value

75.4 ns ± 0.0782 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1956, -0.6899, -0.1991],
               [-0.3943, -0.8755,  0.3890]]),
    x: Batch(
           c: tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1956, -0.6899, -0.1991],
        [-0.3943, -0.8755,  0.3890]])

In [11]:
%timeit b.a

67.4 ns ± 0.0586 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8191, -0.5526, -1.0943],
               [ 0.9378,  1.4598, -1.1956]]),
    x: Batch(
           c: tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078]]),
       ),
)

In [13]:
%timeit b.a = new_value

548 ns ± 0.454 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

892 ns ± 2.99 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

150 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

151 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe1d6547e50>
├── 'a' --> tensor([[[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]],
│           
│                   [[ 1.1956, -0.6899, -0.1991],
│                    [-0.3943, -0.8755,  0.3890]]])
└── 'x' --> <FastTreeValue 0x7fe1d6547c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe0ecbb1a30>
├── 'a' --> tensor([[ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890],
│                   [ 1.1956, -0.6899, -0.1991],
│                   [-0.3943, -0.8755,  0.3890]])
└── 'x' --> <FastTreeValue 0x7fe0ed19e5b0>
    └── 'c' --> tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                        [ 0.7860,  0.4375,  0.5272, -1.3228],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.8 µs ± 73.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5220,  0.0430,  0.3161,  0.5389],
                       [ 0.7860,  0.4375,  0.5272, -1.3228],
                       [ 1.3343, -0.8959,  1.2044,  2.3078]],
              
                      [[-0.5220,  0.0430,  0.3161,  0.5389],
                       [ 0.7860,  0.4375,  0.5272, -1.3228],
                       [ 1.3343, -0.8959,  1.2044,  2.3078]],
              
                      [[-0.5220,  0.0430,  0.3161,  0.5389],
                       [ 0.7860,  0.4375,  0.5272, -1.3228],
                       [ 1.3343, -0.8959,  1.2044,  2.3078]],
              
                      [[-0.5220,  0.0430,  0.3161,  0.5389],
                       [ 0.7860,  0.4375,  0.5272, -1.3228],
                       [ 1.3343, -0.8959,  1.2044,  2.3078]],
              
                      [[-0.5220,  0.0430,  0.3161,  0.5389],
                       [ 0.7860,  0.4375,  0.5272, -1.3228],
                       [ 1.3343, -0.8959,  1.2044,  2.3078]],

In [26]:
%timeit Batch.stack(batches)

83.9 µs ± 416 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078],
                      [-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078],
                      [-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078],
                      [-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078],
                      [-0.5220,  0.0430,  0.3161,  0.5389],
                      [ 0.7860,  0.4375,  0.5272, -1.3228],
                      [ 1.3343, -0.8959,  1.2044,  2.3078],
                      [-0.5220,  0.0430,  0.3161,  0.5389],
                   

In [28]:
%timeit Batch.cat(batches)

154 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

378 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
